# IE48B HW3
## Muhammed Burak Gur
### 2016403198

### Method:

Started with choosing timeseries datasets whose training set at least 100 observations. 

#### Data Sets

1. Plane 
2. Fish
3. PowerCons
4. ECG200
5. Trace

After chosing datasets; which representations, which distance measures and which k-nn classifier are used is given below. 

#### *Notes*:
1. raw time series has no tuning parameter, so the number of combinations is reduced 60(intead 72).
2. Fused lasso has also not any tuning parameter, Piecewise agregate approximation is added.


**3 representations:**
1. raw
2. tree (max_depth=5, max_depth=7)
3. fusedlasso
4. PAA (segment_length = 5)

**4 distance measures:**

1. Euclidian
2. Dynamic Time Warping
3. Longest Common SubSequence
4. Edit Distance on Real Sequences

**3 k-nn classifier setting**

1. k=1 
2. k=3
3. k=5

2 tuning parameters of representations

Besides, 10-fold crossvalidation with 5 repeats is must.

In this notebook consists of test results for the best alternative option of all dataset.
In addition to this notebook, i also provide 5 additional notebook which includes the results of all combination for all dataset.
The results are readed in this test notebook to find best combination. The best combination is used to predict for each dataset's testset. 



In [1]:
require(data.table)
require(ggplot2)
require(TSrepr)
require(TSdist)
require(dtw)
require(Rcpp)

Loading required package: data.table

Warning message:
"package 'data.table' was built under R version 4.0.5"
Loading required package: ggplot2

Warning message:
"package 'ggplot2' was built under R version 4.0.5"
Loading required package: TSrepr

Warning message:
"package 'TSrepr' was built under R version 4.0.5"
Loading required package: TSdist

Warning message:
"package 'TSdist' was built under R version 4.0.5"
Loading required package: proxy

Warning message:
"package 'proxy' was built under R version 4.0.5"

Attaching package: 'proxy'


The following objects are masked from 'package:stats':

    as.dist, dist


The following object is masked from 'package:base':

    as.matrix


Registered S3 method overwritten by 'quantmod':
  method            from
  as.zoo.data.frame zoo 

Loaded TSdist v3.7. See ?TSdist for help, citation("TSdist") for use in publication.


Loading required package: dtw

Warning message:
"package 'dtw' was built under R version 4.0.5"
Loaded dtw v1.22-3. See ?

In [86]:
nn_classify_cv=function(dist_matrix,train_class,test_indices,k=1){
    
    test_distances_to_train=dist_matrix[test_indices,]
    test_distances_to_train=test_distances_to_train[,-test_indices]
    train_class=train_class[-test_indices]
    #print(str(test_distances_to_train))
    ordered_indices=apply(test_distances_to_train,1,order)
    if(k==1){
        nearest_class=as.numeric(testclass[as.numeric(ordered_indices[1,])])
        nearest_class=data.table(id=test_indices,nearest_class)
    } else {
        nearest_class=apply(ordered_indices[1:k,],2,function(x) {testclass[x]})
        nearest_class=data.table(id=test_indices,t(nearest_class))
    }
    
    long_nn_class=melt(nearest_class,'id')

    class_counts=long_nn_class[,.N,list(id,value)]
    class_counts[,predicted_prob:=N/k]
    wide_class_prob_predictions=dcast(class_counts,id~value,value.var='predicted_prob')
    wide_class_prob_predictions[is.na(wide_class_prob_predictions)]=0
    class_predictions=class_counts[,list(predicted=value[which.max(N)]),by=list(id)]
    
    
    return(list(prediction=class_predictions,prob_estimates=wide_class_prob_predictions))
    
}

## Power Cons

In [94]:
current_folder=getwd()
result_path <- sprintf('%s/overall_summary',current_folder)
result_path
summary_files <- list.files(result_path)

[1] "C:/Users/brkmb/Desktop/kitaplar/IE48B/HW3/overall_summary"

In [95]:
result_file <- fread(sprintf('%s/overall_summary/%s',current_folder,summary_files[4]))
head(result_file)

approach,k,avg_acc,sdev_acc,result_count
<chr>,<int>,<dbl>,<dbl>,<int>
PowerCons_dtw_raw_dist.csv,1,0.6800000,0.10442466,50
PowerCons_dtw_raw_dist.csv,3,0.7044444,0.11304441,50
PowerCons_dtw_raw_dist.csv,5,0.7033333,0.09759646,50
PowerCons_dtw_raw_lasso_dist.csv,1,0.4800000,0.09441443,50
PowerCons_dtw_raw_lasso_dist.csv,3,0.5288889,0.09864284,50
PowerCons_dtw_raw_lasso_dist.csv,5,0.5200000,0.08891723,50


In [96]:
result_file[which.max(result_file$avg_acc)]

approach,k,avg_acc,sdev_acc,result_count
<chr>,<int>,<dbl>,<dbl>,<int>
PowerCons_euc_raw_dist.csv,5,0.8055556,0.1078022,50


In [97]:
# assuming you have the data folder in your working directory in the following format:
# 'working_directory/ClassificationData/dataset_name/'
current_folder=getwd()
dataset='PowerCons'
main_path=sprintf('%s/ClassificationData/%s/%s',current_folder,dataset,dataset)

dist_path=sprintf('%s/ClassificationData/%s/distances/%s',current_folder,dataset,dataset)

train_data_path=sprintf('%s_TRAIN.txt',main_path)
traindata=as.matrix(fread(train_data_path))
traindata_rep = fread(train_data_path) ## for representations, class data needs to be hold.

test_data_path=sprintf('%s_TEST.txt',main_path)
testdata=as.matrix(fread(test_data_path))
testdata_rep = fread(test_data_path)
#first column is the class variable
trainclass=traindata[,1] # takes -1 and 1
testclass=testdata[,1] # takes -1 and 1

#drop first column
traindata=traindata[,2:ncol(traindata)]
testdata=testdata[,2:ncol(testdata)]

# instance characteristics
tlength=ncol(traindata)
n_series_train=nrow(traindata)
n_series_test=nrow(testdata)

In [98]:
# raw + euclidian
large_number = 10000
dist_euc=as.matrix(dist(testdata))
diag(dist_euc)=large_number
head(dist_euc)

,1,2,3,4,5,6,7,8,9,10,...,171,172,173,174,175,176,177,178,179,180
1,10000.00000,12.602579,12.222774,13.248400,13.615873,16.71695,14.644688,16.194326,16.934902,15.596282,...,18.19275,20.75004,20.64529,15.76431,18.85436,22.89062,21.48077,26.26043,22.74638,22.97886
2,12.60258,10000.000000,8.727004,5.632184,11.239849,17.20037,9.480522,11.269880,12.439272,11.512532,...,13.06302,20.62899,22.09126,15.01097,18.95199,20.13869,18.49111,25.10112,24.18594,24.35061
3,12.22277,8.727004,10000.000000,7.363226,9.174715,13.56591,10.076244,9.600708,9.450349,8.591403,...,10.43739,15.23881,18.27731,11.08157,14.68532,15.97152,14.32664,20.34550,19.71487,19.79172
4,13.24840,5.632184,7.363226,10000.000000,9.171210,15.40824,7.849955,8.710620,8.806021,8.885826,...,11.66464,18.20504,21.42165,12.97808,16.52674,18.01430,14.68868,23.07513,21.94212,22.66183
5,13.61587,11.239849,9.174715,9.171210,10000.000000,13.15536,7.959617,9.061953,10.698986,9.271214,...,12.86526,17.83980,21.76029,14.13778,15.57255,17.82080,13.88937,22.28154,21.72317,22.73815
6,16.71695,17.200372,13.565906,15.408241,13.155364,10000.00000,14.502927,11.482500,16.426354,11.994766,...,16.94492,18.94307,23.27470,14.13037,17.17638,17.36839,17.68232,22.58707,21.56828,21.52150


In [102]:
# cv indices start here
require(TunePareto)

set.seed(15011998)
nof_rep=5
n_fold=10
cv_indices=generateCVRuns(testclass, ntimes =nof_rep, nfold = n_fold, 
                          leaveOneOut = FALSE, stratified = TRUE)

str(cv_indices)


List of 5
 $ Run  1:List of 10
  ..$ Fold  1 : int [1:18] 1 70 21 136 71 73 24 141 75 96 ...
  ..$ Fold  2 : int [1:18] 10 25 85 4 122 149 125 23 26 115 ...
  ..$ Fold  3 : int [1:18] 128 11 143 65 15 28 63 131 146 169 ...
  ..$ Fold  4 : int [1:18] 74 138 88 129 89 135 86 5 139 99 ...
  ..$ Fold  5 : int [1:18] 130 140 20 8 148 6 145 150 127 33 ...
  ..$ Fold  6 : int [1:18] 134 12 83 82 76 17 2 14 13 103 ...
  ..$ Fold  7 : int [1:18] 78 147 121 133 22 3 81 19 62 59 ...
  ..$ Fold  8 : int [1:18] 79 67 123 77 61 132 126 9 137 91 ...
  ..$ Fold  9 : int [1:18] 30 7 16 90 27 72 144 69 29 154 ...
  ..$ Fold  10: int [1:18] 64 87 80 142 18 68 84 66 124 47 ...
 $ Run  2:List of 10
  ..$ Fold  1 : int [1:18] 5 148 144 18 133 142 76 124 67 53 ...
  ..$ Fold  2 : int [1:18] 69 75 65 21 137 10 7 70 127 169 ...
  ..$ Fold  3 : int [1:18] 135 138 90 25 24 68 28 83 1 151 ...
  ..$ Fold  4 : int [1:18] 147 2 19 149 150 123 63 3 23 111 ...
  ..$ Fold  5 : int [1:18] 132 88 121 29 122 79 134 22 143

In [103]:
k_levels=c(5)
result=vector('list',1*nof_rep*n_fold*length(k_levels))
iter=1
dist_mat=as.matrix(dist_euc)
    for(i in 1:nof_rep){
        this_fold=cv_indices[[i]]
        for(j in 1:n_fold){
            test_indices=this_fold[[j]]
            for(k in 1:length(k_levels)){
                current_k=k_levels[k]
                current_fold=nn_classify_cv(dist_mat,testclass,test_indices,k=current_k)
                accuracy=sum(testclass[test_indices]==current_fold$prediction$predicted)/length(test_indices)
                tmp=data.table(approach='PowerCons_euc_raw_dist',repid=i,foldid=j,
                               k=current_k,acc=accuracy)
                result[[iter]]=tmp
                iter=iter+1
                
            }
            
        }
    
    }   
    


In [104]:
overall_results=rbindlist(result)
overall_results[,list(avg_acc=mean(acc),sdev_acc=sd(acc),result_count=.N),by=list(approach,k)]

powercon_test <- overall_results[,list(avg_acc=mean(acc),sdev_acc=sd(acc),result_count=.N),by=list(approach,k)]


approach,k,avg_acc,sdev_acc,result_count
<chr>,<dbl>,<dbl>,<dbl>,<int>
PowerCons_euc_raw_dist,5,0.8533333,0.08381193,50


## ECG200

In [61]:
current_folder=getwd()
result_path <- sprintf('%s/overall_summary',current_folder)
result_path
summary_files <- list.files(result_path)

[1] "C:/Users/brkmb/Desktop/kitaplar/IE48B/HW3/overall_summary"

In [63]:
result_file <- fread(sprintf('%s/overall_summary/%s',current_folder,summary_files[1]))
head(result_file)

approach,k,avg_acc,sdev_acc,result_count
<chr>,<int>,<dbl>,<dbl>,<int>
ECG200_dtw_raw_dist.csv,1,0.608,0.1440521,50
ECG200_dtw_raw_dist.csv,3,0.616,0.1447870,50
ECG200_dtw_raw_dist.csv,5,0.648,0.1358871,50
ECG200_dtw_raw_lasso_dist.csv,1,0.586,0.1340271,50
ECG200_dtw_raw_lasso_dist.csv,3,0.576,0.1436549,50
ECG200_dtw_raw_lasso_dist.csv,5,0.642,0.1263135,50


In [64]:
result_file[which.max(result_file$avg_acc)]

approach,k,avg_acc,sdev_acc,result_count
<chr>,<int>,<dbl>,<dbl>,<int>
ECG200_euc_raw_lasso_dist.csv,5,0.676,0.09160697,50


In [88]:
# assuming you have the data folder in your working directory in the following format:
# 'working_directory/ClassificationData/dataset_name/'
current_folder=getwd()
dataset='ECG200'
main_path=sprintf('%s/ClassificationData/%s/%s',current_folder,dataset,dataset)

dist_path=sprintf('%s/ClassificationData/%s/distances/%s',current_folder,dataset,dataset)

train_data_path=sprintf('%s_TRAIN.txt',main_path)
traindata=as.matrix(fread(train_data_path))
traindata_rep = fread(train_data_path) ## for representations, class data needs to be hold.

test_data_path=sprintf('%s_TEST.txt',main_path)
testdata=as.matrix(fread(test_data_path))
testdata_rep = fread(test_data_path)
#first column is the class variable
trainclass=traindata[,1] # takes -1 and 1
testclass=testdata[,1] # takes -1 and 1

#drop first column
traindata=traindata[,2:ncol(traindata)]
testdata=testdata[,2:ncol(testdata)]

# instance characteristics
tlength=ncol(traindata)
n_series_train=nrow(traindata)
n_series_test=nrow(testdata)

### Lasso Representation

In [89]:
setnames(testdata_rep,'V1','class')

testdata_rep <- testdata_rep[order(class)]

head(testdata_rep)

testdata_rep[,class:=as.character(class)]
testdata_rep[,id:=1:.N]
long_test=melt(testdata_rep,id.vars=c('id','class'))

long_test[,time:=as.numeric(gsub("\\D", "", variable))-1]

long_test=long_test[,list(id,class,time,value)]
long_test=long_test[order(id,time)]



class,V2,V3,V4,V5,V6,V7,V8,V9,V10,...,V88,V89,V90,V91,V92,V93,V94,V95,V96,V97
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,...,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
-1,0.4436209,0.9472850,1.924084,2.159790,1.4994468,0.9645078,0.2232562,-0.4249456,0.1212787,...,0.9005108,1.2786880,2.3178780,2.9162543,2.2119475,2.03475300,2.0713024000,1.2541780,1.0378788,1.00961550
-1,0.7956747,0.9502404,1.146668,1.749125,2.4258016,2.6709641,2.4624665,2.0670781,1.6233579,...,0.4303212,0.4871072,0.5797414,0.5628337,0.4640931,0.40246792,0.4493048100,0.5580822,0.5685786,0.40117688
-1,1.3983408,3.1540817,3.437721,2.305671,0.1894286,1.1478420,1.0036473,0.5406101,0.9441429,...,-0.2272631,-0.0607361,-0.6444121,-0.3285149,-0.1655815,-0.58693754,-0.2585569700,-0.4527981,-0.3107281,-0.20251214
-1,-1.6681931,-2.0533347,-2.235332,-2.173837,-1.7415020,-1.5408022,-1.9629656,-2.2387671,-1.8779410,...,0.2997362,0.3390010,0.2185765,0.3177845,0.3205212,0.02183141,0.0003893154,0.4377837,0.5666975,-0.02925205
-1,1.0235784,1.4785416,2.206047,2.826309,2.4762642,1.6743566,1.0884254,0.4325428,-0.1502338,...,0.9428605,0.7259834,0.8662279,1.0608436,1.0785506,0.97807484,0.8598958300,0.9278556,1.0143007,0.64819166
-1,1.2806801,1.5708310,1.915334,2.336203,2.7350768,2.9284487,2.8112030,2.4532867,2.0378798,...,0.5306808,0.6692706,0.8522855,0.9292795,0.8529851,0.71977403,0.6562816900,0.6835170,0.7292954,0.72617319


In [74]:
require(genlasso)
lasso_results <- vector("list",n_series_test)

for(i in 1:n_series_test){
  selected_dt <- long_test[id == i]
  selected_dt <- as.matrix(selected_dt[,4], rownames=FALSE)
  flasso <- trendfilter(selected_dt, ord=0)
  cv <- cv.trendfilter(flasso)
     
    #print(paste0(i,'.Time Series' ))
    #print(paste0('minimum crossvalidated error:',cv$lambda.min))
    #print(paste0('1se error:',cv$lambda.min))
  lasso_results[[i]] <- predict.genlasso(flasso, cv$lambda.min)
  
}

lasso_dt = vector('list', n_series_test)
for(i in seq(1:n_series_test)){
lasso_dt[[i]] =as.data.table(t(lasso_results[[i]]$fit))
}
lasso_fit_dt = rbindlist(lasso_dt)
head(lasso_fit_dt)

Loading required package: genlasso

Warning message:
"package 'genlasso' was built under R version 4.0.5"
Loading required package: Matrix

Loading required package: igraph

Warning message:
"package 'igraph' was built under R version 4.0.5"

Attaching package: 'igraph'


The following objects are masked from 'package:stats':

    decompose, spectrum


The following object is masked from 'package:base':

    union




Fold 1 ... Fold 2 ... Fold 3 ... Fold 4 ... Fold 5 ... 
Fold 1 ... Fold 2 ... Fold 3 ... Fold 4 ... Fold 5 ... 
Fold 1 ... Fold 2 ... Fold 3 ... Fold 4 ... Fold 5 ... 
Fold 1 ... Fold 2 ... Fold 3 ... Fold 4 ... Fold 5 ... 
Fold 1 ... Fold 2 ... Fold 3 ... Fold 4 ... Fold 5 ... 
Fold 1 ... Fold 2 ... Fold 3 ... Fold 4 ... Fold 5 ... 
Fold 1 ... Fold 2 ... Fold 3 ... Fold 4 ... Fold 5 ... 
Fold 1 ... Fold 2 ... Fold 3 ... Fold 4 ... Fold 5 ... 
Fold 1 ... Fold 2 ... Fold 3 ... Fold 4 ... Fold 5 ... 
Fold 1 ... Fold 2 ... Fold 3 ... Fold 4 ... Fold 5 ... 
Fold 1 ... Fold 2 ... Fold 3 ... Fold 4 ... Fold 5 ... 
Fold 1 ... Fold 2 ... Fold 3 ... Fold 4 ... Fold 5 ... 
Fold 1 ... Fold 2 ... Fold 3 ... Fold 4 ... Fold 5 ... 
Fold 1 ... Fold 2 ... Fold 3 ... Fold 4 ... Fold 5 ... 
Fold 1 ... Fold 2 ... Fold 3 ... Fold 4 ... Fold 5 ... 
Fold 1 ... Fold 2 ... Fold 3 ... Fold 4 ... Fold 5 ... 
Fold 1 ... Fold 2 ... Fold 3 ... Fold 4 ... Fold 5 ... 
Fold 1 ... Fold 2 ... Fold 3 ... Fold 4 ... Fold

V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,...,V87,V88,V89,V90,V91,V92,V93,V94,V95,V96
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,...,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
0.4437506,0.9472850,1.924084,2.159530,1.4994468,0.9645078,0.2232562,-0.4246862,0.1212787,0.8299378,...,0.9005108,1.2786880,2.3178780,2.9159949,2.2119475,2.03501243,2.071042970,1.2541780,1.0378788,1.0097452
0.7958401,0.9502404,1.146668,1.749125,2.4258016,2.6706334,2.4624665,2.0670781,1.6233579,1.0882650,...,0.4306519,0.4871072,0.5794107,0.5628337,0.4640931,0.40279857,0.449304810,0.5580822,0.5682479,0.4013422
1.5392518,3.1540817,3.155899,2.305671,0.4712505,0.9348337,0.9348337,0.7423765,0.7423765,-0.3403173,...,-0.2241083,-0.2241083,-0.3625902,-0.3603446,-0.3603446,-0.36034464,-0.355677555,-0.3556776,-0.3270756,-0.3270756
-1.6689915,-2.0533347,-2.233736,-2.173837,-1.7415020,-1.5423989,-1.9629656,-2.2371704,-1.8779410,-1.4731916,...,0.2997362,0.3374043,0.2201732,0.3177845,0.3189245,0.02183141,0.001986015,0.4377837,0.5651008,-0.0284537
1.0247160,1.4785416,2.206047,2.824034,2.4762642,1.6743566,1.0884254,0.4325428,-0.1502338,-0.3205431,...,0.9428605,0.7282587,0.8662279,1.0608436,1.0762753,0.97807484,0.862171130,0.9278556,1.0120254,0.6493293
1.2813682,1.5708310,1.915334,2.336203,2.7350768,2.9270725,2.8112030,2.4532867,2.0378798,1.6964592,...,0.5320570,0.6692706,0.8522855,0.9279033,0.8529851,0.71977403,0.657657900,0.6835170,0.7279192,0.7268613


In [78]:
# fusedlasso + euclidian
dist_euc_lasso=as.matrix(dist(lasso_fit_dt))
diag(dist_euc_lasso)=large_number
head(dist_euc_lasso)

,1,2,3,4,5,6,7,8,9,10,...,91,92,93,94,95,96,97,98,99,100
1,10000.000000,6.733718,12.95087,13.86873,5.640105,9.484028,12.819046,14.99271,6.578266,12.416629,...,10.632393,8.746328,10.653209,11.261851,6.840006,11.644766,6.087696,10.911652,11.531411,8.397250
2,6.733718,10000.000000,12.48802,14.48894,4.620912,8.525335,13.525067,15.37121,3.853801,12.481298,...,10.497548,8.721355,10.798775,10.840489,2.285064,11.474979,5.118460,10.450897,10.646511,7.479863
3,12.950874,12.488021,10000.00000,13.25146,11.779060,12.163735,5.722745,11.74645,11.430270,4.484099,...,9.091818,9.497251,7.539361,5.219027,12.215795,8.044118,11.176717,7.259378,6.463475,9.661444
4,13.868733,14.488936,13.25146,10000.00000,13.626807,17.634752,9.857158,11.91847,12.503228,13.934737,...,11.575566,11.542841,10.786730,12.521695,15.152159,12.628971,13.172855,13.281009,13.845702,12.867492
5,5.640105,4.620912,11.77906,13.62681,10000.000000,9.228834,11.936185,13.30075,3.928156,11.548447,...,10.252126,8.576474,10.185059,10.814550,4.665264,11.092183,4.413893,10.472925,10.897659,6.703307
6,9.484028,8.525335,12.16374,17.63475,9.228834,10000.000000,14.521761,15.66998,10.258284,12.289794,...,12.607397,12.037416,13.108756,11.873481,7.522938,13.103665,10.369211,11.713702,11.584076,11.304711


In [90]:
# cv indices start here
require(TunePareto)

set.seed(15011998)
nof_rep=5
n_fold=10
cv_indices=generateCVRuns(testclass, ntimes =nof_rep, nfold = n_fold, 
                          leaveOneOut = FALSE, stratified = TRUE)

str(cv_indices)



List of 5
 $ Run  1:List of 10
  ..$ Fold  1 : int [1:10] 1 77 50 95 9 28 100 21 54 64
  ..$ Fold  2 : int [1:10] 16 45 25 94 96 84 32 59 14 55
  ..$ Fold  3 : int [1:10] 4 49 35 46 3 91 47 10 5 33
  ..$ Fold  4 : int [1:10] 72 81 57 70 36 93 80 88 40 53
  ..$ Fold  5 : int [1:10] 71 67 44 38 42 2 11 56 82 85
  ..$ Fold  6 : int [1:10] 7 37 8 22 6 79 19 43 27 58
  ..$ Fold  7 : int [1:10] 83 17 31 30 23 63 76 34 29 66
  ..$ Fold  8 : int [1:10] 97 89 20 62 48 90 61 99 15 39
  ..$ Fold  9 : int [1:10] 75 78 68 24 73 51 13 60 65 87
  ..$ Fold  10: int [1:10] 69 18 41 26 12 74 98 86 52 92
 $ Run  2:List of 10
  ..$ Fold  1 : int [1:10] 93 25 49 71 24 20 22 58 43 33
  ..$ Fold  2 : int [1:10] 3 17 12 73 28 100 37 29 21 98
  ..$ Fold  3 : int [1:10] 62 91 9 69 57 83 42 55 88 92
  ..$ Fold  4 : int [1:10] 70 1 2 45 48 8 72 54 5 64
  ..$ Fold  5 : int [1:10] 44 77 95 7 50 63 86 15 27 76
  ..$ Fold  6 : int [1:10] 32 90 81 26 41 80 99 14 56 39
  ..$ Fold  7 : int [1:10] 74 94 97 96 36 78 85 60

In [91]:
k_levels=c(5)
result=vector('list',1*nof_rep*n_fold*length(k_levels))
iter=1
dist_mat=as.matrix(dist_euc_lasso)
    for(i in 1:nof_rep){
        this_fold=cv_indices[[i]]
        for(j in 1:n_fold){
            test_indices=this_fold[[j]]
            for(k in 1:length(k_levels)){
                current_k=k_levels[k]
                current_fold=nn_classify_cv(dist_mat,testclass,test_indices,k=current_k)
                accuracy=sum(testclass[test_indices]==current_fold$prediction$predicted)/length(test_indices)
                tmp=data.table(approach='ECG200_euc_raw_lasso_dist',repid=i,foldid=j,
                               k=current_k,acc=accuracy)
                result[[iter]]=tmp
                iter=iter+1
                
            }
            
        }
    
    }   
    


In [93]:
overall_results=rbindlist(result)
overall_results[,list(avg_acc=mean(acc),sdev_acc=sd(acc),result_count=.N),by=list(approach,k)]

ECG200_test <- overall_results[,list(avg_acc=mean(acc),sdev_acc=sd(acc),result_count=.N),by=list(approach,k)]


approach,k,avg_acc,sdev_acc,result_count
<chr>,<dbl>,<dbl>,<dbl>,<int>
ECG200_euc_raw_lasso_dist,5,0.534,0.1318781,50


## Trace

In [105]:
current_folder=getwd()
result_path <- sprintf('%s/overall_summary',current_folder)
result_path
summary_files <- list.files(result_path)

[1] "C:/Users/brkmb/Desktop/kitaplar/IE48B/HW3/overall_summary"

In [110]:
result_file <- fread(sprintf('%s/overall_summary/%s',current_folder,summary_files[5]))
head(result_file)

approach,k,avg_acc,sdev_acc,result_count
<chr>,<int>,<dbl>,<dbl>,<int>
Trace_dtw_raw_dist.csv,1,0.308,0.1827678,50
Trace_dtw_raw_dist.csv,3,0.332,0.2054761,50
Trace_dtw_raw_dist.csv,5,0.336,0.2087475,50
Trace_dtw_raw_lasso_dist.csv,1,0.282,0.1223745,50
Trace_dtw_raw_lasso_dist.csv,3,0.330,0.1328648,50
Trace_dtw_raw_lasso_dist.csv,5,0.328,0.1414791,50


In [111]:
result_file[which.max(result_file$avg_acc)]

approach,k,avg_acc,sdev_acc,result_count
<chr>,<int>,<dbl>,<dbl>,<int>
Trace_lcss_raw_epsilon_005.csv,5,0.378,0.1148024,50


In [112]:
# assuming you have the data folder in your working directory in the following format:
# 'working_directory/ClassificationData/dataset_name/'
current_folder=getwd()
dataset='Trace'
main_path=sprintf('%s/ClassificationData/%s/%s',current_folder,dataset,dataset)

dist_path=sprintf('%s/ClassificationData/%s/distances/%s',current_folder,dataset,dataset)

train_data_path=sprintf('%s_TRAIN.txt',main_path)
traindata=as.matrix(fread(train_data_path))
traindata_rep = fread(train_data_path) ## for representations, class data needs to be hold.

test_data_path=sprintf('%s_TEST.txt',main_path)
testdata=as.matrix(fread(test_data_path))
testdata_rep = fread(test_data_path)
#first column is the class variable
trainclass=traindata[,1] # takes -1 and 1
testclass=testdata[,1] # takes -1 and 1

#drop first column
traindata=traindata[,2:ncol(traindata)]
testdata=testdata[,2:ncol(testdata)]

# instance characteristics
tlength=ncol(traindata)
n_series_train=nrow(traindata)
n_series_test=nrow(testdata)

In [113]:
# LCSS
dist_lcss=TSDatabaseDistances(testdata,distance='lcss',epsilon=0.05)
dist_lcss=as.matrix(dist_lcss)
diag(dist_lcss)=large_number
head(dist_lcss)

,1,2,3,4,5,6,7,8,9,10,...,91,92,93,94,95,96,97,98,99,100
1,10000,42,34,73,47,41,45,95,54,136,...,40,45,47,47,41,181,52,102,32,112
2,42,10000,113,125,214,123,148,127,134,38,...,121,207,207,152,106,38,139,126,117,40
3,34,113,10000,77,89,168,93,63,93,27,...,146,116,106,96,97,36,81,54,207,26
4,73,125,77,10000,140,96,191,217,215,115,...,89,129,145,184,99,108,201,190,92,116
5,47,214,89,140,10000,114,178,133,157,45,...,117,221,228,186,121,45,168,133,94,50
6,41,123,168,96,114,10000,108,91,100,38,...,221,128,126,108,168,45,97,89,203,41


In [114]:
# cv indices start here
require(TunePareto)

set.seed(15011998)
nof_rep=5
n_fold=10
cv_indices=generateCVRuns(testclass, ntimes =nof_rep, nfold = n_fold, 
                          leaveOneOut = FALSE, stratified = TRUE)

str(cv_indices)



List of 5
 $ Run  1:List of 10
  ..$ Fold  1 : int [1:10] 1 55 95 88 20 38 46 17 64 48
  ..$ Fold  2 : int [1:10] 26 3 60 66 24 90 97 4 16 45
  ..$ Fold  3 : int [1:10] 11 59 99 93 5 33 67 8 74 28
  ..$ Fold  4 : int [1:10] 31 37 91 56 47 76 86 39 49 10
  ..$ Fold  5 : int [1:10] 29 18 15 19 75 53 34 72 22 30
  ..$ Fold  6 : int [1:10] 14 21 73 62 82 51 43 98 78 87
  ..$ Fold  7 : int [1:10] 63 27 13 58 54 9 32 85 36 23
  ..$ Fold  8 : int [1:10] 96 79 57 70 35 94 80 12 100 83
  ..$ Fold  9 : int [1:10] 50 6 89 2 68 81 44 69 65 52
  ..$ Fold  10: int [1:10] 61 41 92 84 71 77 25 7 40 42
 $ Run  2:List of 10
  ..$ Fold  1 : int [1:10] 1 41 55 24 19 2 12 25 98 52
  ..$ Fold  2 : int [1:10] 37 15 18 5 35 71 80 39 87 65
  ..$ Fold  3 : int [1:10] 29 27 79 93 20 67 94 7 40 16
  ..$ Fold  4 : int [1:10] 6 73 60 47 75 8 81 72 100 36
  ..$ Fold  5 : int [1:10] 26 14 31 89 38 4 51 76 42 78
  ..$ Fold  6 : int [1:10] 61 50 13 88 56 17 43 34 74 45
  ..$ Fold  7 : int [1:10] 91 96 57 62 92 9 86 33 

In [115]:
k_levels=c(5)
result=vector('list',1*nof_rep*n_fold*length(k_levels))
iter=1
dist_mat=as.matrix(dist_lcss)
    for(i in 1:nof_rep){
        this_fold=cv_indices[[i]]
        for(j in 1:n_fold){
            test_indices=this_fold[[j]]
            for(k in 1:length(k_levels)){
                current_k=k_levels[k]
                current_fold=nn_classify_cv(dist_mat,testclass,test_indices,k=current_k)
                accuracy=sum(testclass[test_indices]==current_fold$prediction$predicted)/length(test_indices)
                tmp=data.table(approach='Trace_lcss_raw_epsilon_005',repid=i,foldid=j,
                               k=current_k,acc=accuracy)
                result[[iter]]=tmp
                iter=iter+1
                
            }
            
        }
    
    }   
    


In [116]:
overall_results=rbindlist(result)
overall_results[,list(avg_acc=mean(acc),sdev_acc=sd(acc),result_count=.N),by=list(approach,k)]

Trace_test <- overall_results[,list(avg_acc=mean(acc),sdev_acc=sd(acc),result_count=.N),by=list(approach,k)]


approach,k,avg_acc,sdev_acc,result_count
<chr>,<dbl>,<dbl>,<dbl>,<int>
Trace_lcss_raw_epsilon_005,5,0.216,0.120136,50


## Fish

In [117]:
current_folder=getwd()
result_path <- sprintf('%s/overall_summary',current_folder)
result_path
summary_files <- list.files(result_path)

[1] "C:/Users/brkmb/Desktop/kitaplar/IE48B/HW3/overall_summary"

In [123]:
result_file <- fread(sprintf('%s/overall_summary/%s',current_folder,summary_files[2]))
head(result_file)

approach,k,avg_acc,sdev_acc,result_count
<chr>,<int>,<dbl>,<dbl>,<int>
Fish_dtw_raw_dist.csv,1,0.2069935,0.09354002,50
Fish_dtw_raw_dist.csv,3,0.2125490,0.09287440,50
Fish_dtw_raw_dist.csv,5,0.2272549,0.11030237,50
Fish_dtw_raw_lasso_dist.csv,1,0.1473203,0.07817485,50
Fish_dtw_raw_lasso_dist.csv,3,0.1552288,0.08357654,50
Fish_dtw_raw_lasso_dist.csv,5,0.1691503,0.10900514,50


In [124]:
result_file[which.max(result_file$avg_acc)]

approach,k,avg_acc,sdev_acc,result_count
<chr>,<int>,<dbl>,<dbl>,<int>
Fish_erp_raw_gap_005.csv,5,0.2376471,0.1219087,50


In [125]:
# assuming you have the data folder in your working directory in the following format:
# 'working_directory/ClassificationData/dataset_name/'
current_folder=getwd()
dataset='Trace'
main_path=sprintf('%s/ClassificationData/%s/%s',current_folder,dataset,dataset)

dist_path=sprintf('%s/ClassificationData/%s/distances/%s',current_folder,dataset,dataset)

train_data_path=sprintf('%s_TRAIN.txt',main_path)
traindata=as.matrix(fread(train_data_path))
traindata_rep = fread(train_data_path) ## for representations, class data needs to be hold.

test_data_path=sprintf('%s_TEST.txt',main_path)
testdata=as.matrix(fread(test_data_path))
testdata_rep = fread(test_data_path)
#first column is the class variable
trainclass=traindata[,1] # takes -1 and 1
testclass=testdata[,1] # takes -1 and 1

#drop first column
traindata=traindata[,2:ncol(traindata)]
testdata=testdata[,2:ncol(testdata)]

# instance characteristics
tlength=ncol(traindata)
n_series_train=nrow(traindata)
n_series_test=nrow(testdata)

In [127]:
# ERP 
dist_erp=TSDatabaseDistances(testdata,distance='erp',g=0.5)
dist_erp=as.matrix(dist_erp)
diag(dist_erp)=large_number

head(dist_erp)
  

,1,2,3,4,5,6,7,8,9,10,...,91,92,93,94,95,96,97,98,99,100
1,10000.0000,134.76189,154.50479,114.54812,126.52280,129.09953,135.71716,116.09486,122.85897,38.86499,...,121.42211,124.27134,123.63886,136.37701,104.54270,27.30391,135.18612,127.06520,144.10821,44.86471
2,134.7619,10000.00000,83.58826,51.44008,16.82923,78.34217,49.78171,51.64496,51.09915,116.81555,...,79.67025,18.52168,17.48572,48.91614,85.08042,121.75635,45.95431,53.20548,80.71447,112.55945
3,154.5048,83.58826,10000.00000,80.31223,95.42967,34.58958,63.94953,91.54711,69.21203,130.21316,...,45.27293,90.82412,91.30024,65.57319,66.50950,136.54098,72.35115,103.04936,18.65834,125.36645
4,114.5481,51.44008,80.31223,10000.00000,50.83663,68.89272,28.84965,18.60148,17.51135,93.19252,...,68.58391,49.41968,46.03039,29.50848,71.02986,100.00777,26.35218,39.01221,73.75224,88.90665
5,126.5228,16.82923,95.42967,50.83663,10000.00000,85.80286,53.77338,47.49790,53.36712,110.80021,...,84.57160,15.33114,12.22592,50.11016,86.53476,114.65945,48.33709,45.68340,90.80299,107.27404
6,129.0995,78.34217,34.58958,68.89272,85.80286,10000.00000,63.36386,78.09243,61.34540,102.27484,...,15.46658,80.82800,80.25583,60.96353,35.03629,109.14831,69.18164,90.92156,20.38468,97.05716


In [129]:
# cv indices start here
require(TunePareto)

set.seed(15011998)
nof_rep=5
n_fold=10
cv_indices=generateCVRuns(testclass, ntimes =nof_rep, nfold = n_fold, 
                          leaveOneOut = FALSE, stratified = TRUE)

str(cv_indices)



List of 5
 $ Run  1:List of 10
  ..$ Fold  1 : int [1:10] 1 55 95 88 20 38 46 17 64 48
  ..$ Fold  2 : int [1:10] 26 3 60 66 24 90 97 4 16 45
  ..$ Fold  3 : int [1:10] 11 59 99 93 5 33 67 8 74 28
  ..$ Fold  4 : int [1:10] 31 37 91 56 47 76 86 39 49 10
  ..$ Fold  5 : int [1:10] 29 18 15 19 75 53 34 72 22 30
  ..$ Fold  6 : int [1:10] 14 21 73 62 82 51 43 98 78 87
  ..$ Fold  7 : int [1:10] 63 27 13 58 54 9 32 85 36 23
  ..$ Fold  8 : int [1:10] 96 79 57 70 35 94 80 12 100 83
  ..$ Fold  9 : int [1:10] 50 6 89 2 68 81 44 69 65 52
  ..$ Fold  10: int [1:10] 61 41 92 84 71 77 25 7 40 42
 $ Run  2:List of 10
  ..$ Fold  1 : int [1:10] 1 41 55 24 19 2 12 25 98 52
  ..$ Fold  2 : int [1:10] 37 15 18 5 35 71 80 39 87 65
  ..$ Fold  3 : int [1:10] 29 27 79 93 20 67 94 7 40 16
  ..$ Fold  4 : int [1:10] 6 73 60 47 75 8 81 72 100 36
  ..$ Fold  5 : int [1:10] 26 14 31 89 38 4 51 76 42 78
  ..$ Fold  6 : int [1:10] 61 50 13 88 56 17 43 34 74 45
  ..$ Fold  7 : int [1:10] 91 96 57 62 92 9 86 33 

In [132]:
k_levels=c(5)
result=vector('list',1*nof_rep*n_fold*length(k_levels))
iter=1
dist_mat=as.matrix(dist_erp)
    for(i in 1:nof_rep){
        this_fold=cv_indices[[i]]
        for(j in 1:n_fold){
            test_indices=this_fold[[j]]
            for(k in 1:length(k_levels)){
                current_k=k_levels[k]
                current_fold=nn_classify_cv(dist_mat,testclass,test_indices,k=current_k)
                accuracy=sum(testclass[test_indices]==current_fold$prediction$predicted)/length(test_indices)
                tmp=data.table(approach='Fish_erp_raw_gap_005',repid=i,foldid=j,
                               k=current_k,acc=accuracy)
                result[[iter]]=tmp
                iter=iter+1
                
            }
            
        }
    
    }   
    


In [133]:
overall_results=rbindlist(result)
overall_results[,list(avg_acc=mean(acc),sdev_acc=sd(acc),result_count=.N),by=list(approach,k)]

Fish_test <- overall_results[,list(avg_acc=mean(acc),sdev_acc=sd(acc),result_count=.N),by=list(approach,k)]


approach,k,avg_acc,sdev_acc,result_count
<chr>,<dbl>,<dbl>,<dbl>,<int>
Fish_erp_raw_gap_005,5,0.34,0.2185294,50


## Plane

In [172]:
current_folder=getwd()
result_path <- sprintf('%s/overall_summary',current_folder)
result_path
summary_files <- list.files(result_path)

[1] "C:/Users/brkmb/Desktop/kitaplar/IE48B/HW3/overall_summary"

In [177]:
result_file <- fread(sprintf('%s/overall_summary/%s',current_folder,summary_files[3]))
head(result_file)

approach,k,avg_acc,sdev_acc,result_count
<chr>,<int>,<dbl>,<dbl>,<int>
Plane_dtw_raw_dist.csv,1,0.2054545,0.1396934,50
Plane_dtw_raw_dist.csv,3,0.2283636,0.1369085,50
Plane_dtw_raw_dist.csv,5,0.2318182,0.1646664,50
Plane_dtw_raw_lasso_dist.csv,1,0.1930909,0.1135240,50
Plane_dtw_raw_lasso_dist.csv,3,0.1969091,0.1260023,50
Plane_dtw_raw_lasso_dist.csv,5,0.2081818,0.1136679,50


In [178]:
result_file[which.max(result_file$avg_acc)]

approach,k,avg_acc,sdev_acc,result_count
<chr>,<int>,<dbl>,<dbl>,<int>
Plane_erp_raw_gap_005.csv,1,0.2363636,0.1213991,50


In [179]:
# assuming you have the data folder in your working directory in the following format:
# 'working_directory/ClassificationData/dataset_name/'
current_folder=getwd()
dataset='Plane'
main_path=sprintf('%s/ClassificationData/%s/%s',current_folder,dataset,dataset)

dist_path=sprintf('%s/ClassificationData/%s/distances/%s',current_folder,dataset,dataset)

train_data_path=sprintf('%s_TRAIN.txt',main_path)
traindata=as.matrix(fread(train_data_path))
traindata_rep = fread(train_data_path) ## for representations, class data needs to be hold.

test_data_path=sprintf('%s_TEST.txt',main_path)
testdata=as.matrix(fread(test_data_path))
testdata_rep = fread(test_data_path)
#first column is the class variable
trainclass=traindata[,1] # takes -1 and 1
testclass=testdata[,1] # takes -1 and 1

#drop first column
traindata=traindata[,2:ncol(traindata)]
testdata=testdata[,2:ncol(testdata)]

# instance characteristics
tlength=ncol(traindata)
n_series_train=nrow(traindata)
n_series_test=nrow(testdata)

In [180]:
# ERP 
dist_erp=TSDatabaseDistances(testdata,distance='erp',g=0.5)
dist_erp=as.matrix(dist_erp)
diag(dist_erp)=large_number

head(dist_erp)
  

,1,2,3,4,5,6,7,8,9,10,...,96,97,98,99,100,101,102,103,104,105
1,10000.00000,36.59676,34.94862,68.87180,31.47556,61.30967,31.23078,29.55560,63.70924,31.92770,...,66.36585,37.50739,35.71251,31.60123,68.41949,66.77601,27.32364,61.71513,61.89477,38.24219
2,36.59676,10000.00000,37.49889,74.25690,29.94068,52.39039,38.78204,29.39735,52.74171,41.72082,...,69.16672,17.71720,13.30515,35.48411,70.48359,69.83359,37.40572,50.74881,69.42518,44.23802
3,34.94862,37.49889,10000.00000,83.56188,34.26557,50.37123,18.81804,35.75646,52.07760,21.48607,...,80.76781,34.27051,32.19023,14.69735,83.05472,80.76318,23.03812,49.13097,77.13465,44.25985
4,68.87180,74.25690,83.56188,10000.00000,68.80185,89.88894,85.99519,68.25810,90.31544,84.62525,...,18.18438,77.16138,76.31238,81.08344,18.87280,16.90053,81.59709,90.20654,19.09714,78.14921
5,31.47556,29.94068,34.26557,68.80185,10000.00000,57.39139,36.02060,11.14709,57.38708,39.49252,...,62.40665,27.52314,25.62572,32.67941,66.39454,63.97857,34.83507,57.43031,58.99004,50.23150
6,61.30967,52.39039,50.37123,89.88894,57.39139,10000.00000,54.63032,59.40209,14.68855,54.18470,...,88.28876,52.21030,52.91752,50.06855,89.42089,88.62063,58.20211,11.51988,88.56126,44.45007


In [181]:
# cv indices start here
require(TunePareto)

set.seed(15011998)
nof_rep=5
n_fold=10
cv_indices=generateCVRuns(testclass, ntimes =nof_rep, nfold = n_fold, 
                          leaveOneOut = FALSE, stratified = TRUE)

str(cv_indices)



List of 5
 $ Run  1:List of 10
  ..$ Fold  1 : int [1:11] 1 19 28 50 76 100 77 48 5 6 ...
  ..$ Fold  2 : int [1:11] 49 27 58 10 85 30 4 21 61 41 ...
  ..$ Fold  3 : int [1:11] 26 70 62 57 47 104 25 81 67 64 ...
  ..$ Fold  4 : int [1:11] 32 72 97 55 29 35 54 79 14 17 ...
  ..$ Fold  5 : int [1:11] 46 89 87 3 71 65 84 42 90 22 ...
  ..$ Fold  6 : int [1:10] 13 94 66 86 73 16 11 51 33 105
  ..$ Fold  7 : int [1:10] 40 24 23 12 36 101 80 8 9 82
  ..$ Fold  8 : int [1:10] 56 98 99 15 91 45 43 60 103 20
  ..$ Fold  9 : int [1:10] 44 93 68 7 59 63 74 95 18 92
  ..$ Fold  10: int [1:10] 53 2 102 37 31 96 75 39 38 69
 $ Run  2:List of 10
  ..$ Fold  1 : int [1:11] 19 1 58 102 71 59 101 5 42 33 ...
  ..$ Fold  2 : int [1:11] 27 49 24 57 91 25 84 39 41 22 ...
  ..$ Fold  3 : int [1:11] 46 98 72 15 76 54 16 95 14 6 ...
  ..$ Fold  4 : int [1:11] 26 28 23 68 50 100 77 75 18 103 ...
  ..$ Fold  5 : int [1:11] 40 2 66 55 73 31 63 60 64 17 ...
  ..$ Fold  6 : int [1:10] 44 89 93 10 3 96 43 51 38 20


In [182]:
k_levels=c(5)
result=vector('list',1*nof_rep*n_fold*length(k_levels))
iter=1
dist_mat=as.matrix(dist_erp)
    for(i in 1:nof_rep){
        this_fold=cv_indices[[i]]
        for(j in 1:n_fold){
            test_indices=this_fold[[j]]
            for(k in 1:length(k_levels)){
                current_k=k_levels[k]
                current_fold=nn_classify_cv(dist_mat,testclass,test_indices,k=current_k)
                accuracy=sum(testclass[test_indices]==current_fold$prediction$predicted)/length(test_indices)
                tmp=data.table(approach='Plane_erp_raw_gap_005',repid=i,foldid=j,
                               k=current_k,acc=accuracy)
                result[[iter]]=tmp
                iter=iter+1
                
            }
            
        }
    
    }   
    


In [183]:
overall_results=rbindlist(result)
overall_results[,list(avg_acc=mean(acc),sdev_acc=sd(acc),result_count=.N),by=list(approach,k)]

plane_test <- overall_results[,list(avg_acc=mean(acc),sdev_acc=sd(acc),result_count=.N),by=list(approach,k)]


approach,k,avg_acc,sdev_acc,result_count
<chr>,<dbl>,<dbl>,<dbl>,<int>
Plane_erp_raw_gap_005,5,0.2298182,0.1851553,50


## Comparing Test Results

In [194]:
whole_test_results <- rbind(Trace_test,Fish_test,ECG200_test,powercon_test,plane_test, colnames = TRUE, fill = TRUE)
whole_test_results <- whole_test_results[1:5,1:4]
whole_test_results[order(avg_acc)]

approach,k,avg_acc,sdev_acc
<chr>,<dbl>,<dbl>,<dbl>
Trace_lcss_raw_epsilon_005,5,0.2160000,0.12013598
Plane_erp_raw_gap_005,5,0.2298182,0.18515528
Fish_erp_raw_gap_005,5,0.3400000,0.21852941
ECG200_euc_raw_lasso_dist,5,0.5340000,0.13187811
PowerCons_euc_raw_dist,5,0.8533333,0.08381193


When we look at the test results, PowerCons dataset is the most accurate one. Its combination is also easy to apply comparing to other methods. 
Besides PowerCons and ECG200, we cannot find any good model for rest to give high accuracy predictions. 